In [129]:
from transformers import *

In [241]:
bert = BertModel.from_pretrained("/Users/HaoShaochun/Documents/Tools/chinese_rbt3_pytorch/")

I0614 00:25:14.133267 4653999552 configuration_utils.py:281] loading configuration file /Users/HaoShaochun/Documents/Tools/chinese_rbt3_pytorch/config.json
I0614 00:25:14.139116 4653999552 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_rep

In [242]:
bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [158]:
model_path = "/Users/HaoShaochun/Documents/Tools/bart"

In [130]:
def shift_tokens_right(input_ids, pad_token_id):
    """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens

In [169]:
def invert_mask(attention_mask):
    assert attention_mask.dim() == 2
    return attention_mask.eq(0)

In [170]:
def make_padding_mask(input_ids, padding_idx=1):
    """True for pad tokens"""
    padding_mask = input_ids.eq(padding_idx)
    if not padding_mask.any():
        padding_mask = None
    return padding_mask

In [204]:
input_ids = torch.LongTensor(([[2, 0, 38, 654, 47, 1, 1, 1]]))
input_ids

tensor([[  2,   0,  38, 654,  47,   1,   1,   1]])

In [205]:
make_padding_mask(input_ids, 1)

tensor([[False, False, False, False, False,  True,  True,  True]])

In [206]:
input_ids = torch.LongTensor(([[2, 0, 38, 654, 47]]))
input_ids
make_padding_mask(input_ids, 1)

In [173]:
def fill_with_neg_inf(t):
    """FP16-compatible function that fills a input_ids with -inf."""
    return t.float().fill_(float("-inf")).type_as(t)

In [222]:
def _prepare_bart_decoder_inputs(
    config, input_ids, decoder_input_ids=None, decoder_padding_mask=None, causal_mask_dtype=torch.float32
):
    """Prepare masks that ignore padding tokens in the decoder and a causal mask for the decoder if
    none are provided. This mimics the default behavior in fairseq. To override it pass in masks.
    Note: this is not called during generation
    """
    pad_token_id = config.pad_token_id
    if decoder_input_ids is None:
        decoder_input_ids = shift_tokens_right(input_ids, pad_token_id)
    bsz, tgt_len = decoder_input_ids.size()
    if decoder_padding_mask is None:
        decoder_padding_mask = make_padding_mask(decoder_input_ids, pad_token_id)
    else:
        decoder_padding_mask = invert_mask(decoder_padding_mask)
    causal_mask = torch.triu(fill_with_neg_inf(torch.zeros(tgt_len, tgt_len)), 1).to(
        dtype=causal_mask_dtype, device=decoder_input_ids.device
    )
    return decoder_input_ids, decoder_padding_mask, causal_mask

In [145]:
bc = BartConfig()

In [146]:
bc.pad_token_id

1

In [159]:
tk = BartTokenizer(os.path.join(model_path, "roberta-large-vocab.json"), 
                   os.path.join(model_path, "roberta-large-merges.txt"))

In [226]:
tk.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>'}

In [230]:
tk.encode(list(tk.special_tokens_map.values()))

[0, 0, 2, 3, 2, 1, 0, 50264, 2]

In [231]:
tk.all_special_ids

[0, 50264, 3, 2, 1]

In [232]:
tk.all_special_tokens

['<s>', '<mask>', '<unk>', '</s>', '<pad>']

In [240]:
bc.activation_function

'gelu'

In [160]:
tk.encode("I love you")

[0, 38, 657, 47, 2]

In [180]:
input_ids = torch.tensor([tk.encode("I love you")])
input_ids

tensor([[  0,  38, 657,  47,   2]])

In [181]:
shift_tokens_right(input_ids, bc.pad_token_id)

tensor([[  2,   0,  38, 657,  47]])

In [234]:
input_ids = torch.LongTensor(([[0, 38, 654, 47, 2, 1, 1]]))
input_ids

tensor([[  0,  38, 654,  47,   2,   1,   1]])

In [235]:
_prepare_bart_decoder_inputs(bc, input_ids)

(tensor([[  2,   0,  38, 654,  47,   2,   1]]),
 tensor([[False, False, False, False, False, False,  True]]),
 tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., -inf, -inf, -inf],
         [0., 0., 0., 0., 0., -inf, -inf],
         [0., 0., 0., 0., 0., 0., -inf],
         [0., 0., 0., 0., 0., 0., 0.]]))

In [1]:
cd ~/Documents/Study/DL-Models/fairseq/

/Users/HaoShaochun/Documents/Study/DL-Models/fairseq


In [3]:
from fairseq.models.bart import BARTModel

I0613 21:51:19.320928 4653999552 file_utils.py:41] PyTorch version 1.5.0 available.
I0613 21:51:22.606090 4653999552 file_utils.py:57] TensorFlow version 2.1.0 available.


In [238]:
bart = BARTModel.from_pretrained(model_path, checkpoint_file='model.pt')

I0613 23:48:19.481202 4653999552 file_utils.py:68] loading archive file /Users/HaoShaochun/Documents/Tools/bart
I0613 23:48:21.747384 4653999552 denoising.py:102] dictionary: 51200 types


In [239]:
bart.eval()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (embed_tokens): Embedding(51201, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): TransformerEncoderLayer(
   

In [103]:
tokens = bart.encode('i love u')
tokens

tensor([   0,  118,  657, 1717,    2])

In [104]:
last_layer_features = bart.extract_features(tokens)

In [105]:
last_layer_features.shape

torch.Size([1, 5, 768])

In [106]:
tokens.shape

torch.Size([5])

In [107]:
tokens = tokens.unsqueeze(0)
tokens.shape

torch.Size([1, 5])

In [109]:
tokens.to(device=bart.device),
prev_output_tokens = tokens.clone()

In [110]:
prev_output_tokens

tensor([[   0,  118,  657, 1717,    2]])

In [111]:
prev_output_tokens[:, 0] = tokens.gather(
    1,(tokens.ne(bart.task.source_dictionary.pad()).sum(dim=1)- 1).unsqueeze(-1),).squeeze()

prev_output_tokens[:, 1:] = tokens[:, :-1]

In [112]:
prev_output_tokens

tensor([[   2,    0,  118,  657, 1717]])

In [113]:
features, extra = bart.model(
            src_tokens=tokens,
            src_lengths=None,
            prev_output_tokens=prev_output_tokens,
            features_only=True,
            return_all_hiddens=True)

In [115]:
features.shape

torch.Size([1, 5, 768])

In [117]:
extra.keys()

dict_keys(['attn', 'inner_states'])

In [120]:
extra['attn'][0].shape

torch.Size([1, 5, 5])

In [125]:
len(extra['inner_states'])

7

In [128]:
bart.args.arch

'bart_base'

In [236]:
bc.vocab_size

50265